In [ ]:
import UWGeodynamics as GEO
from UWGeodynamics import visualisation as vis

In [ ]:
u = GEO.UnitRegistry

In [ ]:
# Characteristic values of the system
half_rate = (1.8 * u.centimeter / u.year).to(u.meter / u.second)
model_length = 60e3 * u.meter
model_height = 60e3 * u.meter
bodyforce = (3300 * u.kilogram / u.metre**3 * 9.81 * u.meter / u.second**2)

KL = model_length
Kt = KL / half_rate
KM = bodyforce * KL**2 * Kt**2

GEO.scaling_coefficients["[length]"] = KL
GEO.scaling_coefficients["[time]"] = Kt
GEO.scaling_coefficients["[mass]"] = KM

In [ ]:
Model = GEO.Model(elementRes=(16,16,16), 
                  minCoord=(-30 * u.kilometer, -30. * u.kilometer, -20 * u.kilometer),
                  maxCoord=( 30 * u.kilometer, 30. * u.kilometer,  10 * u.kilometer))

In [ ]:
Model.outputDir = "1_21_3D_ColumnsTractionBottom"

In [ ]:
air = Model.add_material(name="air", shape=GEO.shapes.Layer3D(top=Model.top, bottom=0.0))
background = Model.add_material(name="background", shape=GEO.shapes.Layer3D(top=air.bottom, bottom=Model.bottom))
heavyColumn = Model.add_material(name="Heavy Column", shape=GEO.shapes.Box(minX=-10.*u.kilometer,
                                                                           maxX=10. * u.kilometer,
                                                                           minY=-10. * u.kilometer,
                                                                           maxY=10. * u.kilometer,
                                                                           top=0, bottom=Model.bottom))

In [ ]:
Fig = vis.Figure()
Fig.Surface(Model.mesh, Model.projMaterialField, cullface=False, opacity=0.5)
viewer = Fig.viewer(resolution=(1200,600))
viewer.window()

In [ ]:
air.density =  0. * u.kilogram / u.metre**3
background.density = 2700. * u.kilogram / u.metre**3
heavyColumn.density = 3300. * u.kilogram / u.metre**3

In [ ]:
import numpy as np
x = GEO.nd(-30 * u.kilometer)
y = GEO.nd(-30 * u.kilometer)
z = GEO.nd(-20 * u.kilometer)
P = Model.lithostatic_pressureField.evaluate(np.array([[x,y,z]]))
bottomPress = GEO.dimensionalise(P, u.megapascal)

In [ ]:
air.viscosity         = 1e19 * u.pascal * u.second
background.viscosity  = 1e25 * u.pascal * u.second
heavyColumn.viscosity = 1e25 * u.pascal * u.second

In [ ]:
Model.set_velocityBCs(left=[0.,0.,0.],
                      right=[0.,0.,0.],
                      front=[0., 0., 0.],
                      back=[0., 0., 0.])

Model.set_stressBCs(bottom=[0.,0., bottomPress])

In [ ]:
Model.run_for(2.0 * u.year, checkpoint_interval=2.0 * u.year)